![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Stop sign violation detection

This notebook is an example how to use DeGirum PySDK to do stop sign violation detection
using object detection, object tracking, zone counting, line cross counting, and event detection.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option.

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify video file name, model name, and other options here

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# stop_zone: zone in which front of the car is detected near the stop line
# stop_line: line that is considered as stop line
hw_location = "@cloud"
model_zoo_url = "degirum/public"
model_name = "yolo_v5n_car_det--512x512_quant_n2x_orca1_1"
video_source = "https://raw.githubusercontent.com/Leonnorblad/DetStopViolation/main/videos/example.mp4"
stop_zone = [
    [[677, 751], [911, 614], [1093, 652], [860, 804]],
]
stop_line = [(860, 804, 1093, 652)]

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools

# load model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=hw_location,
    zoo_url=model_zoo_url,
    token=degirum_tools.get_token(),
    overlay_color=[(255, 0, 0)],
    overlay_line_width=2,
    overlay_show_labels=False,
)

# bbox anchor point for zone and line counters
anchor = degirum_tools.AnchorPoint.BOTTOM_RIGHT

# create object tracker
object_tracker = degirum_tools.ObjectTracker(
    track_thresh=0.35,
    track_buffer=100,
    match_thresh=0.9999,
    trail_depth=20,
    anchor_point=anchor,
)

# create zone counter
window_name = "Display"
zone_counter = degirum_tools.ZoneCounter(
    stop_zone,
    use_tracking=True,
    triggering_position=[anchor],
    window_name=window_name,  # attach display window for interactive zone adjustment
)

# create line crossing counter
line_counter = degirum_tools.LineCounter(
    stop_line,
    anchor,
    accumulate=False,
    show_overlay=True,
    annotation_color=(255, 255, 0),
)

# create event detector: car stops in zone
inzone_detector = degirum_tools.EventDetector(
    """
    Trigger: car_stops_in_zone
    when: ZoneCount
    is greater than: 0
    during: [30, frames]
    for at least: [80, percent]
    """,
    show_overlay=False,
)

# create event detector: car crosses stop line
linecross_detector = degirum_tools.EventDetector(
    """
    Trigger: car_crosses_stopline
    when: LineCount
    is greater than: 0
    during: [1, frame]
    for at least: [1, frame]
    """,
    show_overlay=False,
)

# create event notifier: incorrect stop when car crosses stop line without stopping in zone
annotation_pos = stop_zone[0][0]
incorrect_stop_notifier = degirum_tools.EventNotifier(
    "Incorrect Stop",
    "car_crosses_stopline and not car_stops_in_zone",
    annotation_pos=annotation_pos,
    annotation_color=(255, 0, 0),
    annotation_font_scale=2,
)

# create event notifier: correct stop when car crosses stop line with stopping in zone
correct_stop_notifier = degirum_tools.EventNotifier(
    "Correct Stop",
    "car_crosses_stopline and car_stops_in_zone",
    annotation_pos=annotation_pos,
    annotation_color=(0, 255, 0),
    annotation_font_scale=2,
)

# attach zone counter to model
degirum_tools.attach_analyzers(
    model,
    [
        object_tracker,
        zone_counter,
        inzone_detector,
        line_counter,
        linecross_detector,
        incorrect_stop_notifier,
        correct_stop_notifier,
    ],
)

# run inference and display results
with degirum_tools.Display(window_name) as display:
    for inference_result in degirum_tools.predict_stream(model, video_source):
        display.show(inference_result)

stop_zone = zone_counter._polygons